In [ ]:
#Installing required package to run the Notebook (only needed the very first time of usage)
#!pip install ipywidgets matplotlib tqdm nodejs npm fabio
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!mkdir data results
!mkdir data/MOKE results/MOKE

In [3]:
#Initializing every widgets and variables needed to run XRD plots
import os, glob, time, fabio;
import matplotlib.pyplot as plt;
import numpy as np;
from ipywidgets import interactive, widgets, fixed, Layout, Button, GridBox;
from IPython.display import display;
from tqdm.notebook import tqdm;
from scipy.interpolate import griddata;
from scipy.spatial import QhullError;
from scipy import signal;
from tqdm.notebook import tqdm;

#Widgets for raw and treated MOKE data interactive plots
W_folderpath = widgets.Dropdown(
    options=[('Select a folder', None)]+[(elm, elm) for elm in os.listdir('./data/MOKE/') if not elm.startswith('.')],
    value=None,
    description='Folder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path')
);
W_folderpath_2 = widgets.Dropdown(
    options=[('Select a folder', None)],
    value=None,
    description='Subfolder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path2')
);
W_slider_x = widgets.IntSlider(
    min=-60, max=60,
    step=5,
    description='X position',
    layout=Layout(width='auto', grid_area='X')
);
W_slider_y = widgets.IntSlider(
    min=-40, max=40,
    step=5,
    description='Y position',
    layout=Layout(width='auto', grid_area='Y')
);
W_slider_Xrange = widgets.IntRangeSlider(
    value=[0, 100],
    min=0, max=100,
    step=1,
    description='Time',
    layout=Layout(width='auto', grid_area='Xrange')
);
W_slider_Yrange = widgets.FloatRangeSlider(
    value=[-.5, .5],
    min=-1, max=1,
    step=.01,
    description='Kerr rot.',
    layout=Layout(width='auto', grid_area='Yrange')
);
W_offset = widgets.FloatSlider(
    value=1.0,
    min=-5.0, max=5.0,
    step=0.1,
    description='Sum offset',
    layout=Layout(width='auto', grid_area='offset')
);
W_smooth = widgets.Checkbox(
    value=True,
    description='Smooth',
    tooltip='Apply or remove smooth function for data',
    indent=True,
    layout=Layout(width='auto', grid_area='smooth')
);
W_slider_smth = widgets.Dropdown(
    options=[(i,i) for i in range(0, 11)],
    value=4,
    disabled=True,
    description='Polyn. order',
    layout=Layout(width='80%', grid_area='smooth_slider')
);
W_average = widgets.Checkbox(
    value=True,
    description='Average',
    tooltip='Average between the different aquisition data',
    disabled=False,
    layout=Layout(width='auto', grid_area='average')
);
W_avg_drop = widgets.Dropdown(
    options=[],
    value=None,
    description='Aquisition n°',
    disabled=True,
    layout=Layout(width='85%', grid_area='avg_drop')
);
W_bgr_type = widgets.Dropdown(
    options=[('None', 1)],
    value=1,
    description='Background',
    disabled=True,
    layout=Layout(width='auto', grid_area='bgr_type')
);
W_map_file = widgets.Text(
    value = None,
    description='Filename',
    layout=Layout(width='auto', grid_area='map_file')
);
W_start_button = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    tooltip='Create a map file containing loops, coercivity and reflectivity values',
    layout=Layout(width='auto', grid_area='start_button')
);
W_data_type = widgets.Dropdown(
    options=[()],
    value=None,
    description='Data to plot',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_type')
);
W_plot_dim = widgets.Dropdown(
    options=[('2D map plot',1)],
    value=1,
    description='Plot type',
    disabled=True,
    layout=Layout(width='100%', grid_area='plot_dim')
);
W_data_file = widgets.Dropdown(
    options=[()],
    value=None,
    description='Datafile',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_file')
);
W_tabs = widgets.Tab();
tab_names = ['Raw MOKE data', '2D Data maps'];
W_tabs.children = [widgets.Text(description='filler1'),
                   widgets.Text(description='filler2')];
for i, elm in enumerate(tab_names):
    W_tabs.set_title(i, str(elm));

output = widgets.Output();

#defining all functions for the raw plots (1st tab)
def getBoundaries(folderpath, folderpath_2):
    if folderpath is not None and folderpath_2 is not None:
        fullpath = f'./data/MOKE/{folderpath}/{folderpath_2}/info.txt';
        try:
            with open(fullpath, 'r', encoding='iso-8859-1') as file_info :
                for line in file_info:
                    if 'Pulse_voltage' in line:
                        pulse_volt = line.split('=')[1];
                    elif 'Average_per_point' in line:
                        avg_pts = line.split('=')[1];
                    elif 'Number_of_points_x' in line:
                        nb_x = int(line.split('=')[1]);
                    elif 'Number_of_points_y' in line:
                        nb_y = int(line.split('=')[1]);
                    elif 'Scanning_dimension_x' in line:
                        length_x = float(line.split('=')[1]);
                    elif 'Scanning_dimension_y' in line:
                        length_y = float(line.split('=')[1]);
                    elif 'shape' in line:
                        shape = line.split('=')[1].strip();
                return pulse_volt, avg_pts, nb_x, nb_y, length_x, length_y, shape;
        except FileNotFoundError:
            print(f'info.txt file not found (required to plot data)');
            return None;
    return None;

def initialize_sliders(slider_x, slider_y, nb_x, nb_y, length_x, length_y, shape):
    if shape == 'CIRCLE':
        step = int(length_x/nb_x);
        max = int(step*(nb_x-1)/2); min = -max;

        W_slider_x.step, W_slider_y.step = step, step;
        W_slider_x.min, W_slider_y.min = min, min;
        W_slider_x.max, W_slider_y.max = max, max;
        return min, min, max, max, step, step;
    elif shape == 'RECTANGULAR':
        step_x = int(length_x/(nb_x-1)); step_y = int(length_y/(nb_y-1));
        max_x = int(step_x*(nb_x-1)/2); min_x = -max_x;
        max_y = int(step_y*(nb_y-1)/2); min_y = -max_y;
        
        W_slider_x.step, W_slider_y.step = step_x, step_y;
        W_slider_x.min, W_slider_y.min = min_x, min_y;
        W_slider_x.max, W_slider_y.max = max_x, max_y;
        return min_x, min_y, max_x, max_y, step_x, step_y;
    else:
        print('not rect or circle');
        return None;
    return 0;

def createFilename(folderpath, folderpath_2, slider_x, slider_y):
    '''converting coordinates to indexes in Areamap00x00y.ext'''
    if folderpath is None or folderpath_2 is None:
        print(f"Start by selecting a folder with your data.");
        return None;
    fullpath = f'./data/MOKE/{folderpath}/{folderpath_2}/';
    
    mag = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_magnetization.txt';
    pul = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_pulse.txt';
    sum = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_sum.txt';
    data_mag = f'{fullpath}/{mag}';
    data_pul = f'{fullpath}/{pul}';
    data_sum = f'{fullpath}/{sum}';

    count_check = 0;
    for file in os.listdir(fullpath):
        if count_check == 3:
            break;
        elif file.endswith(mag):
            data_mag = f'{fullpath}/{file}';
            count_check += 1;
        elif file.endswith(pul):
            data_pul = f'{fullpath}/{file}';
            count_check += 1;
        elif file.endswith(sum):
            data_sum = f'{fullpath}/{file}';
            count_check += 1;
    return data_mag, data_pul, data_sum;

def getDataRange(average, avg_pts):
    data_range = range(int(avg_pts));
    if not average:
        W_avg_drop.options=[(f'Plot {i}',i) for i in data_range];
        W_avg_drop.disabled=False;
        if W_avg_drop.value is None:
            W_avg_drop.value = 0;
            return None;
        data_range = [W_avg_drop.value];
    else :
        W_avg_drop.disabled=True;
    return data_range;

def readFromFiles(data_mag, data_pul, data_sum, data_range):
    mag_values = []; pul_values = []; sum_values = [];
    time_step = 0.05 #microsecondes (or 50 ns)
    
    try:
        with open(#opening the 3 raw datafiles from MOKE output
            data_mag, 'r', encoding='iso-8859-1') as file_mag, open(
            data_pul, 'r', encoding='iso-8859-1') as file_pul, open(
            data_sum, 'r', encoding='iso-8859-1') as file_sum:

            for line in file_mag:
                if not line.startswith('D'):#skipping the headers
                    data = line.split();
                    if len(data) > 0 :
                        mag_values.append(np.mean([float(data[i]) for i in data_range]));
            for line in file_pul:
                if not line.startswith('P'):
                    data = line.split();
                    if len(data) > 0 :
                        pul_values.append(np.mean([float(data[i]) for i in data_range]));
            for line in file_sum:
                if not line.startswith('S'):
                    data = line.split();
                    if len(data) > 0 :
                        sum_values.append(np.mean([float(data[i]) for i in data_range]));
                        
        t_values = [j*time_step for j in range(len(mag_values))];
        return mag_values, pul_values, sum_values, t_values;
    except FileNotFoundError :
        print("Datafile not found, you are probably outside the range");
        return None, None, None, None;

def calculate_field_adjusted(pulse_data, adjust_to_max, max_field=4):
    '''Integrate pulse data to get field data'''
    field = np.cumsum(pulse_data);
    # Calculate the scale factor differently
    if adjust_to_max:
        scale_factor = max_field/np.abs(np.min(field));
    else:
        scale_factor = max_field/np.abs(np.max(field));
    field *= -scale_factor;
    
    return field;

def adjust_magnetization(magnetization_data, ranges, avg_sum):
    # Calculate Mr for specified ranges and adjust magnetization data
    Mr = [np.mean(magnetization_data[range[0]-1:range[1]+1]) for range in ranges];

    return np.concatenate([(magnetization_data[:1002] - np.mean(Mr[:2]))/avg_sum,
                           (magnetization_data[1002:] - np.mean(Mr[2:]))/avg_sum]);

def calculateFieldValues(mag_values, pul_values, sum_values, pulse_volt):
    max_field_c = 0.92667*int(pulse_volt)/100;
    avg_sum = np.mean(sum_values);

    field_values = np.concatenate([calculate_field_adjusted(pul_values[:1002], True, max_field=max_field_c),
                                   calculate_field_adjusted(pul_values[1002:], False, max_field=max_field_c)]);
    corr_mag_values = adjust_magnetization(mag_values, [(50, 250), (750, 950), (1050, 1250), (1750, 1950)], avg_sum);
    
    return field_values, corr_mag_values;

def smoothData(field_values, corr_mag_values, smooth, polyorder):
    if smooth:
        field_values=signal.savgol_filter(field_values, 12, 10-polyorder);
        corr_mag_values=signal.savgol_filter(corr_mag_values, 12, 10-polyorder);
        W_slider_smth.disabled = False;
    else:
        W_slider_smth.disabled = True;
    return field_values, corr_mag_values;

def extract_coercivity(field_values, corr_mag_values):
    coercivity1, coercivity2 = [], [];
    min_1 = np.min(np.abs(corr_mag_values[356:654]));
    min_2 = np.min(np.abs(corr_mag_values[1356:1664]));
    
    coercivity1 = field_values[np.where(corr_mag_values == min_1)[0]];
    if len(coercivity1) == 0:
        coercivity1 = field_values[np.where(corr_mag_values == -min_1)[0]];
    coercivity2 = field_values[np.where(corr_mag_values == min_2)[0]];
    if len(coercivity2) == 0:
        coercivity2 = field_values[np.where(corr_mag_values == -min_2)[0]];
    return np.mean([np.min(np.abs(coercivity1)), np.min(np.abs(coercivity2))]);

def readAllDataFiles(folderpath, folderpath_2, pulse_volt, average, avg_pts, smooth, polyorder):
    fullpath = f'./data/MOKE/{folderpath}/{folderpath_2}/';
    DATA_LIST = [];
    data_range = getDataRange(average, avg_pts);
    
    for file in tqdm(sorted(os.listdir(fullpath))):
        #break;
        if 'magnetization' in file:
            data_mag = fullpath + file;
            data_pul = fullpath + file.replace('magnetization','pulse');
            data_sum = fullpath + file.replace('magnetization','sum');
            
            #retriving data from files
            file_list = [pos.translate({ord(i): None for i in 'xy'}) for pos in file.split('_')[1:3]];
            mag_values, pul_values, sum_values, t_values = readFromFiles(data_mag, data_pul, data_sum, data_range);
                    
            #calculate applied field from pulse values
            field_values, corr_mag_values = calculateFieldValues(mag_values, pul_values, sum_values, pulse_volt);
            field_value, corr_mag_values = smoothData(field_values, corr_mag_values, smooth, polyorder);

            #exacting all the processed parameters
            coercivity = extract_coercivity(field_values, corr_mag_values);
            reflectivity = np.mean(sum_values);
            
            file_list.append(coercivity);
            file_list.append(reflectivity);
            DATA_LIST.append(file_list);
    save_path = f'results/MOKE/{folderpath_2}_MOKE.dat';
    with open(save_path, 'w') as file:
        file.write(f'x_pos\ty_pos\tCoercivity (T)\tReflectivity (V)\n');
        for line in DATA_LIST:
            x_pos = float(line[0]); y_pos = float(line[1]);
            if np.abs(x_pos) + np.abs(y_pos) <= 60 and np.abs(x_pos) <= 40 and np.abs(y_pos) <= 40:
                file.write(f'{line[0]}\t{line[1]}\t{line[2]}\t{line[3]}\n');
    print(f'File created ./{save_path}');
    return(DATA_LIST);

def raw_plot(folderpath, folderpath_2, average, avg_drop,
             slider_x, slider_y, smooth, polyorder,
             slider_Xrange, slider_Yrange, offset):
    '''Plotting function checking the state of every widget and plot the corresponding graph'''
    global SAVE_PLOT_MULT;
    try:
        #fetching all the inputs from user
        pulse_volt, avg_pts, nb_x, nb_y, length_x, length_y, shape = getBoundaries(folderpath, folderpath_2);
        initialize_sliders(slider_x, slider_y, nb_x, nb_y, length_x, length_y, shape);
    except TypeError:
        print("Select a folder to start");
        return 1;
    data_mag, data_pul, data_sum = createFilename(folderpath, folderpath_2, slider_x, slider_y);
    data_range = getDataRange(average, avg_pts);
    if data_range is None:
        return 0;
    mag_values, pul_values, sum_values, t_values = readFromFiles(data_mag, data_pul, data_sum, data_range);
    if mag_values is None:
        return 1;
    field_values, corr_mag_values = calculateFieldValues(mag_values, pul_values, sum_values, pulse_volt);
    field_value, corr_mag_values = smoothData(field_values, corr_mag_values, smooth, polyorder);
    coercivity = extract_coercivity(field_values, corr_mag_values);
    
    plt.close('all'); 
    #plotting the figures
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6));
    ax1.set_title(f'Raw MOKE data');
    ax1.set_xlabel('Duration (μs)');
    ax1.set_ylabel('Kerr rotation (a.u)');
    ax1.plot(t_values, mag_values, linewidth = 0.9, color='blue', label="magnetization");
    ax1.plot(t_values, pul_values, linewidth = 0.9, color='red', label="pulse");
    ax1.plot(t_values, [y/10 for y in field_values], linewidth = 0.9, color='green', label="field");
    ax1.plot(t_values, [sums-offset for sums in sum_values], linewidth = 0.9, color='orange', label="sum");
    ax1.set_xlim(slider_Xrange);
    ax1.set_ylim(slider_Yrange);
    ax1.legend(loc='upper right');
    ax1.grid(True);
        
    ax2.set_title(f'Loop extracted from raw data');
    ax2.plot(field_values[356:654], corr_mag_values[356:654], linestyle='None', marker='o', color='None', markeredgecolor='blue');
    ax2.plot(field_values[1356:1664], corr_mag_values[1356:1664], linestyle='None', marker='o', color='None', markeredgecolor='blue',
             label=' Coercivity = {:.2f} T'.format(coercivity));
    ax2.legend(loc='upper left');
    ax2.grid(True);
        
    plt.tight_layout();
    plt.figure(dpi=400);
    plt.show();
    return 0;

def build_2D_maps(folderpath, folderpath_2, average, smooth, polyorder, start):
    try:
        pulse_volt, avg_pts, nb_x, nb_y, length_x, length_y, shape = getBoundaries(folderpath, folderpath_2);
    except TypeError:
        print("Select a folder to start");
        return 1;

    if start == True:
        W_start_button.value = False;
        W_start_button.disabled = True;
        DATA_LIST = readAllDataFiles(folderpath, folderpath_2, pulse_volt, average, avg_pts, smooth, polyorder);
        W_start_button.disabled = False;
    
    if f'{folderpath_2}_MOKE.dat' in os.listdir('./results/MOKE/'):
        fullpath = f'./results/MOKE/{folderpath_2}_MOKE.dat';
        with open(fullpath) as datafile:
            header = (next(datafile)).strip('\n').split('\t');
            data = np.genfromtxt(datafile, delimiter='\t', skip_header=0);
            
        plot_number = 2;
        plot_number_2 = 3;
        print(header)
        
        X_data = data[:, 0];
        Y_data = data[:, 1];
        Z_data = data[:, plot_number];
        Z_2data = data[:, plot_number_2];

        X, Y, Z, Z_2 = np.array([]), np.array([]), np.array([]), np.array([]);
        for i in range(len(X_data)):
            X = np.append(X, X_data[i]);
            Y = np.append(Y, Y_data[i]);
            Z = np.append(Z, Z_data[i]);
            Z_2 = np.append(Z_2, Z_2data[i]);
            
        xi = np.linspace(X.min(), X.max(), 500);
        yi = np.linspace(Y.min(), Y.max(), 500);
        #interpolation between the points for 2D map
        try:
            zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='cubic');
            zi_2 = griddata((X, Y), Z_2, (xi[None,:], yi[:,None]), method='cubic');
        except QhullError:
            print(f"Error: Data is not 2-dimensionnal, cannot create a map");
            return 1;
            
        # Create the contour plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        CS = ax1.contourf(xi, yi, zi, 100, cmap=plt.cm.rainbow, vmax=max(Z), vmin=min(Z));
        fig.colorbar(CS, ax=ax1);
        CS_2 = ax2.contourf(xi, yi, zi_2, 100, cmap=plt.cm.rainbow, vmax=max(Z_2), vmin=min(Z_2));
        fig.colorbar(CS_2, ax=ax2);
            
        ax1.set_title(f'{header[plot_number]}');
        ax2.set_title(f'{header[plot_number+1]}');
                    
        plt.tight_layout();
        plt.figure(dpi=400);
        plt.show();
    else :
        print('Datafile not yet generated, press Start to build the file.')
    return 0;

def redirectFromTab(folderpath, folderpath_2, average, avg_drop,
                    slider_x, slider_y, smooth, polyorder,
                    slider_Xrange, slider_Yrange, offset,
                    start):
    '''checking which tab is currently selected by the user'''
    #start_time = time.time();

    match W_tabs.selected_index:
        case 0 :
            raw_plot(folderpath, folderpath_2, average, avg_drop,
                     slider_x, slider_y, smooth, polyorder,
                     slider_Xrange, slider_Yrange, offset);
        case 1 :
            build_2D_maps(folderpath, folderpath_2, average, smooth, polyorder, start);
    #print(f"Execution time (system) is {time.time()-start_time:.3f} s");
    return 0;

#global variables
SAVE_PLOT_MULT = [];

#interactive function needed to check an input from the user
widget = interactive(
    redirectFromTab,
    folderpath=W_folderpath,
    folderpath_2=W_folderpath_2,
    average=W_average,
    avg_drop=W_avg_drop,
    slider_x=W_slider_x,
    slider_y=W_slider_y,
    smooth=W_smooth,
    polyorder=W_slider_smth,
    slider_Xrange=W_slider_Xrange,
    slider_Yrange=W_slider_Yrange,
    offset=W_offset,
    start=W_start_button,
    continuous_update=False
    );

#defining the different layouts of each interactive plots
raw_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto auto',
    grid_template_columns='16.66% 16.66% 16.66% 16.66% 16.66% 16.66%',
    grid_template_areas='''
    "path path X X Xrange Xrange"
    "path2 path2 Y Y Yrange Yrange"
    "average avg_drop smooth smooth_slider offset offset"
    "plot plot plot plot plot plot"
    '''
);
build_2D_maps_layout = Layout(
    width='100%',
    grid_template_rows='auto auto auto',
    grid_template_columns='16.66% 16.66% 16.67% 25% 25%',
    grid_template_areas='''
    "path path start_button . ."
    "path2 path2 start_button . ."
    "plot plot plot plot plot"
    '''
);
#defining the gridboxes
raw_gridbox = GridBox(
    children=[W_folderpath, W_folderpath_2, W_average, W_avg_drop,
              W_slider_x, W_slider_y, W_smooth, W_slider_smth,
              W_slider_Xrange, W_slider_Yrange, W_offset,
              widget.children[-1]],
    layout=raw_plot_layout);

maps_2D_gridbox = GridBox(
    children=[W_folderpath, W_folderpath_2, W_start_button, widget.children[-1]],
    layout=build_2D_maps_layout);

#adding the plot to the display
widget.children[-1].layout=Layout(width='auto', grid_area='plot');

#tab menu including all the different plots
tab_children = [raw_gridbox, maps_2D_gridbox];
W_tabs.children = tab_children;
display(W_tabs, output);

def on_tab_change(change):
    with output:
        index = W_tabs.selected_index;
        if index == 0:
            W_slider_Xrange.value = [0, 100];
        if index == 1:
            W_slider_Xrange.value = [0, 99];
                    
def on_folder_change(change):
    with output:
        if W_folderpath.value is not None:
            W_folderpath_2.options=[(elm, elm) for elm in os.listdir(f'./data/MOKE/{W_folderpath.value}') if not elm.startswith('.')];
        else:
            W_folderpath_2.options=[('Select a folder', None)];
            W_folderpath_2.value = None;
    return 0;
    
W_tabs.observe(on_tab_change, names='selected_index');
W_folderpath.observe(on_folder_change, names='value');

Output()

@end-of-script